# An Analysis for face anti-spoofing of 20 different architectures in CNNs.

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten
from keras.applications.resnet import ResNet50, ResNet101, preprocess_input
from keras.applications import (
    InceptionV3, ResNet50, ResNet101, MobileNetV2,
    DenseNet121, VGG16, VGG19, DenseNet169,
    DenseNet201, Xception, NASNetLarge, NASNetMobile,
    EfficientNetB0, EfficientNetB1, EfficientNetB2,
    EfficientNetB3, EfficientNetB4, EfficientNetB5,
    EfficientNetB6, EfficientNetB7
)
from keras.preprocessing import image
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import (
    f1_score, precision_score, recall_score,
    confusion_matrix, classification_report, roc_curve
)
import matplotlib.pyplot as plt

## Prepare datasets for training, validation and Test set

In [ ]:

# Set dataset paths
ct_train_path = "/data/Desktop//trainset/"
ct_validation_path = "/data/Desktop/validation/"
ct_test_path = "/data/Desktop/testset/"

# Number of classes depends on the dataset (2 for the binary classification)
ct_num_classes = 2

# Image size depends on the pretrained model (224 for InceptionV3)
np.random.seed(42)
image_size = 224
batch_size = 16
epoch_num = 15

# Define a data generator for training set with data augmentation
ct_train_datagen = image.ImageDataGenerator(
    rescale=1./255,
    vertical_flip=True,
    horizontal_flip=True,
    preprocessing_function=preprocess_input)

ct_train_generator = ct_train_datagen.flow_from_directory(
    ct_train_path,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='categorical')

# Define a data generator for validation set (without data augmentation)
ct_validation_datagen = image.ImageDataGenerator(rescale=1./255, preprocessing_function=preprocess_input)

ct_validation_generator = ct_validation_datagen.flow_from_directory(
    ct_validation_path,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='categorical')

# Define a data generator for testing set (without data augmentation)
ct_test_datagen = image.ImageDataGenerator(rescale=1./255, preprocessing_function=preprocess_input)

ct_test_generator = ct_test_datagen.flow_from_directory(
    ct_test_path,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='categorical')

# Importing 20 pre-trained CNN models 

In [ ]:

# Assuming image_size is set to 224
input_shape = (224, 224, 3)  # Adjust according to the target size of your images

# Create different models for comparison and choose the best model
#base_model = InceptionV3(include_top=False, pooling='max', weights='imagenet', input_shape=input_shape)
#base_model = ResNet50(include_top=False, pooling='max', weights='imagenet', input_shape=input_shape)
#base_model = ResNet101(include_top=False, pooling='max', weights='imagenet', input_shape=input_shape)
#base_model = MobileNetV2(include_top=False, pooling='max', weights='imagenet', input_shape=input_shape)
#base_model = DenseNet121(include_top=False, pooling='max', weights='imagenet', input_shape=input_shape)
#base_model = VGG16(include_top=False, pooling='max', weights='imagenet', input_shape=input_shape)
#base_model = VGG19(include_top=False, pooling='max', weights='imagenet', input_shape=input_shape)
#base_model = DenseNet169(include_top=False, pooling='max', weights='imagenet', input_shape=input_shape)
#base_model = DenseNet201(include_top=False, pooling='max', weights='imagenet', input_shape=input_shape)
#base_model = Xception(include_top=False, pooling='max', weights='imagenet', input_shape=input_shape)
#base_model = NASNetLarge(include_top=False, weights='imagenet', input_shape=input_shape)
#base_model = NASNetMobile(include_top=False, weights='imagenet', input_shape=input_shape)
#base_model = EfficientNetB0(include_top=False, weights='imagenet', input_shape=input_shape)
#base_model = EfficientNetB1(include_top=False, weights='imagenet', input_shape=input_shape)
#base_model = EfficientNetB2(include_top=False, weights='imagenet', input_shape=input_shape)
#base_model = EfficientNetB3(include_top=False, weights='imagenet', input_shape=input_shape)
#base_model = EfficientNetB4(include_top=False, weights='imagenet', input_shape=input_shape)
#base_model = EfficientNetB5(include_top=False, weights='imagenet', input_shape=input_shape)
#base_model = EfficientNetB6(include_top=False, weights='imagenet', input_shape=input_shape)
#base_model = EfficientNetB7(include_top=False, weights='imagenet', input_shape=input_shape)

In [ ]:

# Assuming image_size is set to 224
input_shape = (224, 224, 3)  # Adjust according to the target size of your images

# Create the EfficientNetB7 base (without the top layer)
base_model = EfficientNetB7(include_top=False, pooling='max', weights='imagenet', input_shape=input_shape)

# Freeze the weights of the EfficientNetB7 base
base_model.trainable = False

# Create the top layers for binary classification
top_model = Sequential()
top_model.add(Flatten())
top_model.add(Dense(64, activation='relu'))
top_model.add(Dense(ct_num_classes, activation='sigmoid'))  # Change to 'sigmoid' for binary classification

# Combine the base and top models
model = Sequential()
model.add(base_model)
model.add(top_model)

# Compile the combined model
opt = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()


In [ ]:
# Define model parameters
steps_per_epoch = ct_train_generator.samples // batch_size
validation_steps = ct_validation_generator.samples // batch_size

In [ ]:
# Assuming you have already defined ct_train_generator, ct_validation_generator, steps_per_epoch, epoch_num, and validation_steps

# Train the model using the fit method
history = model.fit(
    ct_train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epoch_num,
    validation_data=ct_validation_generator,
    validation_steps=validation_steps,
    verbose=1
)
# Plot training and validation curves
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:

# Assuming you have the true labels for the validation and test sets
true_labels_validation = ct_validation_generator.classes
true_labels_test = ct_test_generator.classes

# Assuming you have the predicted probabilities for the positive class (spoofed) from your model
predicted_probabilities_validation = model.predict(ct_validation_generator)[:, 1]
predicted_probabilities_test = model.predict(ct_test_generator)[:, 1]

# Compute the ROC curve on the validation set
fpr, tpr, thresholds = roc_curve(true_labels_validation, predicted_probabilities_validation, pos_label=1)

# Calculate EER on the validation set
eer = fpr[np.nanargmin(np.abs(fpr - (1 - tpr)))]

# Find the threshold where FAR equals FRR (EER threshold)
eer_threshold = thresholds[np.nanargmin(np.abs(fpr - (1 - tpr)))]

# Apply the EER threshold to the test set predictions
predictions_test = (predicted_probabilities_test > eer_threshold).astype(int)

# Calculate HTER on the test set
false_acceptance = np.sum((predictions_test == 1) & (true_labels_test == 0))
false_rejection = np.sum((predictions_test == 0) & (true_labels_test == 1))
total_samples_test = len(true_labels_test)

hter = (false_acceptance + false_rejection) / (2 * total_samples_test)

# Display EER and HTER
print(f"Equal Error Rate (EER) on validation set: {eer:.2%}")
print(f"Half Total Error Rate (HTER) for Face Anti-Spoofing: {hter:.2%}")